In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import regex as re


In [10]:
def start_browser():
    browser = webdriver.Chrome()
    browser.get('https://x.com/i/flow/login?lang=es')
    browser.maximize_window()
    return browser


In [11]:
def scrapping(browser, correo, username, password, busquedas):
    df = pd.DataFrame({'Source': [], 'Target': []})
    wait = WebDriverWait(browser, 90)

    # Hacer log-in en twitter
    boton_correo = wait.until(EC.presence_of_element_located((By.XPATH,
    '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[4]/label/div/div[2]/div/input')))
    boton_correo.send_keys(correo)

    boton_next = wait.until(EC.element_to_be_clickable((By.XPATH,
    '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]')))
    boton_next.click()

    try:
        boton_username = wait.until(EC.presence_of_element_located((By.XPATH,
        '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div[2]/label/div/div[2]/div/input')))
        boton_username.send_keys(username)

        boton_next2 = wait.until(EC.element_to_be_clickable((By.XPATH,
        '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/button')))
        boton_next2.click()
    except Exception:
        pass

    boton_password = wait.until(EC.presence_of_element_located((By.XPATH,
    '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input')))
    boton_password.send_keys(password)

    boton_login = wait.until(EC.element_to_be_clickable((By.XPATH,
    '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/div/button')))
    boton_login.click()

    # Acepta las cookies
    boton_cookies = wait.until(EC.element_to_be_clickable((By.XPATH,
    '//*[@id="layers"]/div/div/div/div/div/div[2]/button[1]')))
    boton_cookies.click()

    for busqueda in busquedas:
        # Hacer búsqueda con una palabra clave
        browser.get('https://x.com/search?q=' + busqueda + '&src=typed_query')
        
        tweets_distintos = []
        
        # Recolecta tweets
        while len(tweets_distintos) < 20:
            
            browser.execute_script("window.scrollBy({ top: 800, behavior: 'smooth' })")
            time.sleep(2)
            try:
                tweets = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article[@data-testid='tweet']//div[@data-testid='tweetText']")))
            except:
                continue
            
            for tweet in tweets:
                try:
                    tweet_link = tweet.find_element(By.XPATH,
                    "//div[@class='css-175oi2r r-18u37iz r-1wbh5a2 r-1ez5h0i']//div[@class='css-175oi2r r-18u37iz r-1q142lx']//a[@role='link']").get_attribute('href')
                    autor = re.search(r'https://x.com/(.*)/status/.*', tweet_link).group(1)
                    if tweet_link not in tweets_distintos:
                        tweets_distintos.append(tweet_link)
                        browser.get(tweet_link)

                        # Obtiene las respuestas de los tweets
                        respuestas = []
                        last_height = browser.execute_script("return document.body.scrollHeight")
                        while True:
                            time.sleep(3)
                            usernames = browser.find_elements(By.XPATH,
                            "//article[@data-testid='tweet']//div[@class='css-175oi2r r-zl2h9q']//div[@class='css-175oi2r r-18u37iz r-1wbh5a2 r-1ez5h0i']//span[@class='css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3']")
                            respuestas.extend([name_div.text[1:] for name_div in usernames if name_div.text[0] == '@'])
                            browser.execute_script("window.scrollBy(0, document.body.scrollHeight)")
                            new_height = browser.execute_script("return document.body.scrollHeight")
                            if new_height == last_height:
                                break
                            last_height = new_height
                        respuestas = list(set(respuestas))
                            
                        df2 = pd.DataFrame({'usuario1': autor, 'usuario2': respuestas})
                        df = pd.concat([df, df2], ignore_index=True)
                        
                        browser.back()
                except Exception:
                    pass

    return df



In [12]:
correo = 'pruebatwiter2@gmail.com'
username = 'pruebatwiter02'
password = 'passpruebatwiter2'

equipos = [
    '"Alavés"',
    '"Athletic Club"',
    '"Atlético de Madrid"',
    '"FC Barcelona"',
    '"Celta de Vigo"',
    '"Espanyol"',
    '"Getafe CF"',
    '"Girona FC"',
    '"UD Las Palmas"',
    '"CD Leganés"',
    '"Real Mallorca"',
    '"Osasuna"',
    '"Rayo Vallecano"',
    '"Real Betis"',
    '"Real Madrid"',
    '"Real Sociedad"',
    '"Real Valladolid"',
    '"Sevilla FC"',
    '"Valencia CF"',
    'Villarreal LaLiga'
]
busquedas = [f'{equipo}' + ' min_replies:10 lang:es' for equipo in equipos]
busquedas

['"Alavés" min_replies:10 lang:es',
 '"Athletic Club" min_replies:10 lang:es',
 '"Atlético de Madrid" min_replies:10 lang:es',
 '"FC Barcelona" min_replies:10 lang:es',
 '"Celta de Vigo" min_replies:10 lang:es',
 '"Espanyol" min_replies:10 lang:es',
 '"Getafe CF" min_replies:10 lang:es',
 '"Girona FC" min_replies:10 lang:es',
 '"UD Las Palmas" min_replies:10 lang:es',
 '"CD Leganés" min_replies:10 lang:es',
 '"Real Mallorca" min_replies:10 lang:es',
 '"Osasuna" min_replies:10 lang:es',
 '"Rayo Vallecano" min_replies:10 lang:es',
 '"Real Betis" min_replies:10 lang:es',
 '"Real Madrid" min_replies:10 lang:es',
 '"Real Sociedad" min_replies:10 lang:es',
 '"Real Valladolid" min_replies:10 lang:es',
 '"Sevilla FC" min_replies:10 lang:es',
 '"Valencia CF" min_replies:10 lang:es',
 'Villarreal LaLiga min_replies:10 lang:es']

In [ ]:
browser = start_browser()
df = scrapping(browser, correo, username, password, busquedas)
browser.quit()

In [ ]:
df.to_csv('red_social.csv', index=False)